In [ ]:
import tensorflow as tf
import tensorflow.feature_column as fc 

import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from random import shuffle
 

In [ ]:
tf.enable_eager_execution()

In [ ]:


file_name = "x_82_ETF_FOREX_5MIN_RETONLY.csv"
train_df = pandas.read_csv(file_name, header = 0, nrows = 100000)
# test_df = pandas.read_csv(test_file, header = None, names = census_dataset._CSV_COLUMNS)

train_df.drop(["Date"], axis = 1, inplace=True)

In [ ]:
feature_columns = [j for j in train_df.columns if j != "SPY"]
train_df.head()


In [ ]:
def easy_input_function(df, label_key, num_epochs, shuffle, batch_size):
  label = df[label_key]
  ds = tf.data.Dataset.from_tensor_slices((dict(df),label))

  if shuffle:
    ds = ds.shuffle(10000)

  ds = ds.batch(batch_size).repeat(num_epochs)

  return ds

In [ ]:
ds = easy_input_function(train_df, label_key='SPY', num_epochs=5, shuffle=True, batch_size=10)

for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys())[:5])
  print()
  print('A batch of AUDUSD  :', feature_batch['AUDUSD'])
  print()
  print('A batch of Labels:', label_batch )

In [ ]:
#inspect source code of a function/class
# import inspect
# print(inspect.getsource(census_dataset.input_fn))

In [ ]:
import functools

train_inpf = functools.partial(easy_input_function, df = train_df, label_key='SPY', num_epochs=2, shuffle=True, batch_size=64)
# test_inpf = functools.partial(census_dataset.input_fn, test_file, num_epochs=1, shuffle=False, batch_size=64)

In [ ]:
feature_col = [tf.feature_column.numeric_column(k) for k in feature_columns]

In [ ]:
classifier = tf.estimator.LinearClassifier(feature_col)
classifier.train(train_inpf)
# result = classifier.evaluate(test_inpf)

# clear_output()  # used for display in notebook
print(result)

In [ ]:
# result = classifier.evaluate(test_inpf)

In [ ]:
import tensorflow as tf
import tensorflow.feature_column as fc 

import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from random import shuffle
 

In [ ]:
# tf.enable_eager_execution()
2**20

In [ ]:

data_input = ['{0:020b}'.format(i) for i in range(2**15)]
shuffle(data_input)
data_input = [map(int,i) for i in data_input]
ti  = []
for i in data_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
data_input = ti
len(data_input)

In [ ]:
data_output = []
 
for i in data_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    data_output.append(temp_list)

In [ ]:
data_input = np.array(data_input)
data_output = np.array(data_output)
print("i.shape:", data_input.shape, "o.shape:", data_output.shape)

In [ ]:
NUM_EXAMPLES = 10000
test_input = data_input[NUM_EXAMPLES:]
test_output = data_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = data_input[:NUM_EXAMPLES]
train_output = data_output[:NUM_EXAMPLES] #till 10,000

In [ ]:
print(test_input.shape)
print(test_output.shape)

print(train_input.shape)
print(train_output.shape)

In [ ]:
tf.reset_default_graph()
data = tf.placeholder(tf.float32, [None, 20,1])
target = tf.placeholder(tf.float32, [None, 21])

In [ ]:

num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

In [ ]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

In [ ]:
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [ ]:
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))

In [ ]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

In [ ]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [ ]:

# init_op = tf.initialize_all_variables()

sess = tf.Session()
tf.global_variables_initializer()

In [ ]:
batch_size = 100
no_of_batches = int(len(train_input)/batch_size)
epoch = 50
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
    print("Epoch - ",str(i))
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
sess.close()

In [ ]:
sess.run(model.prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]})